In [2]:
import sys
import pandas as pd
sys.path.append('../utils/')
from utils import get_lib_gdf, get_ct_gdf, assignment_dataframes2
from plotting import creating_foliumn_map
from acs_agg import acs_cleaning, acs_join

In [3]:
# mapbox constants
MAPBOX_TOKEN = ""

#### Create library and census tract geodataframe

In [4]:
lib_file_path = "../data/library_locations.csv"
libs = get_lib_gdf(lib_file_path)

In [5]:
ct_file_path = "../data/census_tract_boundaries.geojson"
cts = get_ct_gdf(ct_file_path)

#### Assign census tracts overlapping with lib 1 mile bufferzone

In [6]:
mile1 = assignment_dataframes2(cts, libs, meters=1609)

/Users/qingyi/Documents/uchicago/job/community_data_fellow/cpl/cpl_project/cdf/notebooks/../utils/utils.py:73: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  library_df.loc[:, 'bufferzone'] = library_df['geometry'].buffer(meters)
/Users/qingyi/Documents/uchicago/job/community_data_fellow/cpl/cpl_project/cdf/notebooks/../utils/utils.py:77: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  boundary_df.loc[:, 'centroid'] = boundary_df.geometry.centroid
/Users/qingyi/Documents/uchicago/job/community_data_fellow/cpl/cpl_project/cdf/notebooks/../utils/utils.py:83: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS bef

In [8]:
len(mile1)

64881

#### Assign census tracts outside original buffer zone
For those census tracts that did not overlap with a bufferzone, expand the buffer and assign them

In [ ]:
remaining = pd.merge(cts, mile1.loc[:, ['namelsad10', 'name']], on='namelsad10', how='left')
remaining = remaining[remaining.isnull().any(axis=1)].reset_index()
remaining = remaining[~remaining.loc[:, 'namelsad10'].isin(['Census Tract 9800','Census Tract 7706.02'])]
remaining = remaining.drop(columns=['index', 'name', 'centroid'])

In [ ]:
mile5 = assignment_dataframes2(remaining, libs, meters=9654) #9654

#### Merge datasets for data visualizations

In [ ]:
joined_df = mile1.append(mile5, ignore_index=True)

In [ ]:
creating_foliumn_map(joined_df, cts)

In [ ]:
acs_filepath = 'https://raw.githubusercontent.com/ssegovba/cpl_project/main/data_bases/census_cook_county_dta.csv'
acs_data = acs_cleaning(acs_filepath)

In [ ]:
agg_df = acs_join(acs_data, joined_df)